# T81-558: Applications of Deep Neural Networks
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

**Kaggle Assignment: **

**Student Name: Jason Walker**

# Overview

## Description
This is one of the projects from the course T81-855: Applications of Deep Learning at Washington University in St. Louis. All students must create a Kaggle account and submit a solution. Once you have submitted your solution entry log into Blackboard (at WUSTL) and submit a single file telling me your Kaggle name on the leaderboard (you do not need to register to Kaggle with your real name). This competition will be visible to the public, so there may be non-student submissions as well as student.

The data set for this competition consists of a number of input columns that should be used to predict a stores sales. This is a regression problem. The inputs are a mixture of discrete and category values. The data set is from a simulation.

## Evaluation

The evaluation pages describes how submissions will be scored and how students should format their submissions. The scores are in RMSE.
Submission Format

For every store in the dataset, submission files should contain a sales volume.

The file should contain a header and have the following format:

```
100000,1.23
100001,1.123
100002,3.332
100003,1.53
...
```

# Data

## Data Description

The data contains data and costs for various office supplies. The data came from a simulation and do not directly correspond to any real-world items. See how well you can predict the cost of an item using the provided data. Feature engineering will likely help you. The *name* column may seem useless at first glance; however, it contains information that you can parse to help your predictions.
File descriptions
```
    id - The identifier/primary key.
    name - The name of this item.
    manufacturer - The manufacturer.
    pack - The number of items in this pack.
    weight - The weight of a pack of these items.
    height - The height of a pack of these items.
    width - The width of a pack of these items.
    length - The length of a pack of these items.
    cost - The cost for this item pack. This is what you are to predict (the target). 
```

# Helpful Functions

You will see these at the top of every module and assignment.  These are simply a set of reusable functions that we will make use of.  Each of them will be explained as the semester progresses.  They are explained in greater detail as the course progresses.  Class 4 contains a complete overview of these functions.

In [1]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        
# This function submits an assignment.  You can submit an assignment as much as you like, only the final
# submission counts.  The paramaters are as follows:
# data - Pandas dataframe output.
# key - Your student key that was emailed to you.
# no - The assignment class number, should be 1 through 1.
# source_file - The full path to your Python or IPYNB file.  This must have "_class1" as part of its name.  
# .             The number must match your assignment number.  For example "_class2" for class assignment #2.
def submit(data,key,no,source_file=None):
    if source_file is None and '__file__' not in globals(): raise Exception('Must specify a filename when a Jupyter notebook.')
    if source_file is None: source_file = __file__
    suffix = '_class{}'.format(no)
    if suffix not in source_file: raise Exception('{} must be part of the filename.'.format(suffix))
    with open(source_file, "rb") as image_file:
        encoded_python = base64.b64encode(image_file.read()).decode('ascii')
    ext = os.path.splitext(source_file)[-1].lower()
    if ext not in ['.ipynb','.py']: raise Exception("Source file is {} must be .py or .ipynb".format(ext))
    r = requests.post("https://api.heatonresearch.com/assignment-submit",
        headers={'x-api-key':key}, json={'csv':base64.b64encode(data.to_csv(index=False).encode('ascii')).decode("ascii"),
        'assignment': no, 'ext':ext, 'py':encoded_python})
    if r.status_code == 200:
        print("Success: {}".format(r.text))
    else: print("Failure: {}".format(r.text))

# Kaggle Code



In [2]:
import os
import re
import sklearn

import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn import metrics
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split, KFold
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import regularizers

path = './data'

filename_test = os.path.join(path,"test.csv")
filename_train = os.path.join(path,"train.csv")
filename_sample = os.path.join(path,"sample.csv")
filename_submit = os.path.join(path,"submit.csv")
filename_checkpoint = os.path.join(path,"checkpoint.hdf5")

df_train = pd.read_csv(filename_train,na_values=['NA','?'])

np.random.seed(42) # Uncomment this line to get the same shuffle each time
df_train = df_train.reindex(np.random.permutation(df_train.index))
df_train.reset_index(inplace=True, drop=True)

# Encode Features

# Remove all rows where the specified column is +/- sd standard deviations
def extract_and_encode_features(df):
    color_regex='(?P<color>red|blue|green|yellow|orange|pink|black|brown|white)'
    df['color'] = df.name.str.extract(color_regex, flags=re.IGNORECASE, expand=False)

    quality_regex='(?P<quality>generic|high\squality)'
    df['quality'] = df.name.str.extract(quality_regex, flags=re.IGNORECASE, expand=False)

    size_regex='(?P<size>tiny|small|medium|large)'
    df['size'] = df.name.str.extract(size_regex, flags=re.IGNORECASE, expand=False)

    item_regex='(?P<item>paperclips|paperweights|ink\spens|pencils|stapler|tablets|thumbtacks|post\sit\snotes)'
    df['item'] = df.name.str.extract(item_regex, flags=re.IGNORECASE, expand=False)
    
    for column in ['pack','weight','height','width','length']:
        missing_median(df,column)
    
    #df.insert(1,'surface_area',(df['height']*df['width']*df['length']).astype(int))
    
    ## encode numeric features
    #for column in ['pack','weight','height','width','length','surface_area']:
    #    encode_numeric_zscore(df,column)

    # encode text/categorical features
    for column in ['manufacturer','color','quality','size','item']:
        encode_text_dummy(df,column)
  
extract_and_encode_features(df_train)

ids_train = df_train['id']
df_train.drop('id',1,inplace=True)

names_train = df_train['name']
df_train.drop('name',1,inplace=True)

x,y = to_xy(df_train,'cost')

# Used before KFold
#x_train, x_test, y_train, y_test = train_test_split(    
#    x, y, test_size=0.25, random_state=45)


/Users/jwalker/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


In [ ]:
# Simple function to evaluate the coefficients of a regression
%matplotlib inline    
from IPython.display import display, HTML    

def report_coef(names,coef,intercept):
    r = pd.DataFrame( { 'coef': coef, 'positive': coef>=0  }, index = names )
    r = r.sort_values(by=['coef'])
    display(r)
    print("Intercept: {}".format(intercept))
    r['coef'].plot(kind='barh', color=r['positive'].map({True: 'b', False: 'r'}))
    
# Create linear regression
regressor = sklearn.linear_model.LinearRegression()

# Fit/train linear regression
regressor.fit(x_train,y_train)
# Predict
pred = regressor.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

names = list(df_train.columns.values)
names.remove("cost")
print(names)
report_coef(
  names,
  regressor.coef_[0,:],
  regressor.intercept_)

In [ ]:


import sklearn
from sklearn.linear_model import Lasso

# Create linear regression
regressor = Lasso(random_state=0,alpha=0.01)

# Fit/train LASSO
regressor.fit(x_train,y_train)
# Predict
pred = regressor.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

names = list(df_train.columns.values)
names.remove("cost")
report_coef(
  names,
  regressor.coef_,
  regressor.intercept_)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

lasso = Lasso(random_state=42)
alphas = np.logspace(-8, 8, 10)

scores = list()
scores_std = list()

n_folds = 3

for alpha in alphas:
    lasso.alpha = alpha
    this_scores = cross_val_score(lasso, x, y, cv=n_folds, n_jobs=1)
    scores.append(np.mean(this_scores))
    scores_std.append(np.std(this_scores))

scores, scores_std = np.array(scores), np.array(scores_std)

plt.figure().set_size_inches(8, 6)
plt.semilogx(alphas, scores)

# plot error lines showing +/- std. errors of the scores
std_error = scores_std / np.sqrt(n_folds)

plt.semilogx(alphas, scores + std_error, 'b--')
plt.semilogx(alphas, scores - std_error, 'b--')

# alpha=0.2 controls the translucency of the fill color
plt.fill_between(alphas, scores + std_error, scores - std_error, alpha=0.2)

plt.ylabel('CV score +/- std error')
plt.xlabel('alpha')
plt.axhline(np.max(scores), linestyle='--', color='.5')
plt.xlim([alphas[0], alphas[-1]])

In [ ]:
import sklearn
from sklearn.linear_model import Ridge

# Create linear regression
regressor = Ridge(alpha=1)

# Fit/train Ridge
regressor.fit(x_train,y_train)
# Predict
pred = regressor.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

names = list(df_train.columns.values)
names.remove("cost")

report_coef(
  names,
  regressor.coef_[0,:],
  regressor.intercept_)

In [ ]:
import sklearn
from sklearn.linear_model import ElasticNet

# Create linear regression
regressor = ElasticNet(alpha=0.01, l1_ratio=0.1)

# Fit/train LASSO
regressor.fit(x_train,y_train)
# Predict
pred = regressor.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

names = list(df_train.columns.values)
names.remove("cost")

report_coef(
  names,
  regressor.coef_,
  regressor.intercept_)

In [3]:
# Cross-Validate
kf = KFold(5)

checkpoint = ModelCheckpoint(filepath=filename_checkpoint, verbose=0, save_best_only=True)

# Turn off KFold
#if (0):
oos_y = []
oos_pred = []
fold = 0
    
for train, test in kf.split(x):
    fold+=1
    print("Fold #{}".format(fold))
        
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    model = Sequential()
    model.add(Dense(100, input_dim=x.shape[1], activation='relu')) # Hidden 1
    model.add(Dropout(0.01)) # Dropout Layer
    model.add(Dense(50, activation='relu')) # Hidden 2
    model.add(Dense(25, 
                    kernel_regularizer=regularizers.l2(0.01), #L2 regularization
                    activity_regularizer=regularizers.l1(0.001), #L1 Lasso regularization
                    activation='relu')) # Hidden 3 
    model.add(Dense(10, activation='relu')) # Hidden 4
    model.add(Dense(1)) # Output
    model.compile(loss='mean_squared_error', optimizer='adam')
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=1000, verbose=1, mode='auto')
    model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpoint],verbose=1,epochs=10000)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    oos_pred.append(pred)        

    # Measure this fold's RMSE
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    print("Fold score (RMSE): {}".format(score))


Fold #1
Train on 8000 samples, validate on 2000 samples
Epoch 1/10000
8000/8000 [==============================] - 1s 140us/step - loss: 5175.7872 - val_loss: 4352.9070
Epoch 2/10000
8000/8000 [==============================] - 1s 65us/step - loss: 4924.2017 - val_loss: 3923.2911
Epoch 3/10000
8000/8000 [==============================] - 1s 63us/step - loss: 4585.4411 - val_loss: 3667.3977
Epoch 4/10000
8000/8000 [==============================] - 1s 73us/step - loss: 4383.6586 - val_loss: 3661.4572
Epoch 5/10000
8000/8000 [==============================] - 1s 64us/step - loss: 4246.3518 - val_loss: 3544.9565
Epoch 6/10000
8000/8000 [==============================] - 1s 76us/step - loss: 4228.7725 - val_loss: 3523.5790
Epoch 7/10000
8000/8000 [==============================] - 1s 68us/step - loss: 4188.1532 - val_loss: 3410.1037
Epoch 8/10000
8000/8000 [==============================] - 1s 67us/step - loss: 4034.2734 - val_loss: 3329.7115
Epoch 9/10000
8000/8000 [======================

Epoch 74/10000
8000/8000 [==============================] - 1s 72us/step - loss: 342.9299 - val_loss: 684.2596
Epoch 75/10000
8000/8000 [==============================] - 0s 59us/step - loss: 435.3010 - val_loss: 328.3805
Epoch 76/10000
8000/8000 [==============================] - 0s 56us/step - loss: 325.4734 - val_loss: 235.2390
Epoch 77/10000
8000/8000 [==============================] - 0s 55us/step - loss: 318.8906 - val_loss: 241.4618
Epoch 78/10000
8000/8000 [==============================] - 0s 55us/step - loss: 281.5494 - val_loss: 475.8683
Epoch 79/10000
8000/8000 [==============================] - 0s 52us/step - loss: 296.4854 - val_loss: 264.4123
Epoch 80/10000
8000/8000 [==============================] - 0s 54us/step - loss: 321.0278 - val_loss: 483.0289
Epoch 81/10000
8000/8000 [==============================] - 0s 54us/step - loss: 295.6610 - val_loss: 219.0106
Epoch 82/10000
8000/8000 [==============================] - 0s 54us/step - loss: 328.5895 - val_loss: 330.7988
E

8000/8000 [==============================] - 0s 60us/step - loss: 224.4417 - val_loss: 236.6342
Epoch 148/10000
8000/8000 [==============================] - 1s 64us/step - loss: 300.8182 - val_loss: 173.7771
Epoch 149/10000
8000/8000 [==============================] - 0s 55us/step - loss: 249.8253 - val_loss: 214.7953
Epoch 150/10000
8000/8000 [==============================] - 0s 59us/step - loss: 342.2343 - val_loss: 274.1790
Epoch 151/10000
8000/8000 [==============================] - 0s 55us/step - loss: 261.9254 - val_loss: 186.9095
Epoch 152/10000
8000/8000 [==============================] - 1s 65us/step - loss: 241.4432 - val_loss: 821.7349
Epoch 153/10000
8000/8000 [==============================] - 0s 61us/step - loss: 296.7644 - val_loss: 1077.1799
Epoch 154/10000
8000/8000 [==============================] - 0s 60us/step - loss: 298.9697 - val_loss: 167.2944
Epoch 155/10000
8000/8000 [==============================] - 0s 61us/step - loss: 254.5368 - val_loss: 200.8235
Epoch 1

8000/8000 [==============================] - 0s 52us/step - loss: 235.7236 - val_loss: 327.7357
Epoch 221/10000
8000/8000 [==============================] - 0s 52us/step - loss: 198.7098 - val_loss: 278.3927
Epoch 222/10000
8000/8000 [==============================] - 1s 66us/step - loss: 189.2912 - val_loss: 171.6242
Epoch 223/10000
8000/8000 [==============================] - 0s 54us/step - loss: 225.7318 - val_loss: 171.4298
Epoch 224/10000
8000/8000 [==============================] - 0s 52us/step - loss: 181.6790 - val_loss: 188.8038
Epoch 225/10000
8000/8000 [==============================] - 0s 55us/step - loss: 207.1400 - val_loss: 192.2145
Epoch 226/10000
8000/8000 [==============================] - 0s 54us/step - loss: 167.5697 - val_loss: 252.3629
Epoch 227/10000
8000/8000 [==============================] - 0s 54us/step - loss: 255.8568 - val_loss: 169.4357
Epoch 228/10000
8000/8000 [==============================] - 0s 54us/step - loss: 228.6719 - val_loss: 189.7494
Epoch 22

8000/8000 [==============================] - 0s 56us/step - loss: 177.8844 - val_loss: 382.3879
Epoch 294/10000
8000/8000 [==============================] - 0s 53us/step - loss: 271.3147 - val_loss: 172.3780
Epoch 295/10000
8000/8000 [==============================] - 0s 53us/step - loss: 261.9071 - val_loss: 271.1394
Epoch 296/10000
8000/8000 [==============================] - 0s 52us/step - loss: 218.5575 - val_loss: 256.0332
Epoch 297/10000
8000/8000 [==============================] - 0s 51us/step - loss: 256.0370 - val_loss: 190.3013
Epoch 298/10000
8000/8000 [==============================] - 0s 52us/step - loss: 219.1190 - val_loss: 276.1852
Epoch 299/10000
8000/8000 [==============================] - 0s 53us/step - loss: 407.3614 - val_loss: 165.5791
Epoch 300/10000
8000/8000 [==============================] - 0s 48us/step - loss: 173.6492 - val_loss: 152.2976
Epoch 301/10000
8000/8000 [==============================] - 0s 50us/step - loss: 157.3076 - val_loss: 200.9187
Epoch 30

...

8000/8000 [==============================] - 1s 68us/step - loss: 123.0765 - val_loss: 129.1163
Epoch 2674/10000
8000/8000 [==============================] - 1s 66us/step - loss: 129.6602 - val_loss: 129.8731
Epoch 2675/10000
8000/8000 [==============================] - 1s 66us/step - loss: 145.7110 - val_loss: 134.1748
Epoch 2676/10000
8000/8000 [==============================] - 1s 67us/step - loss: 185.4004 - val_loss: 117.2153
Epoch 2677/10000
8000/8000 [==============================] - 1s 74us/step - loss: 157.5927 - val_loss: 114.4050
Epoch 2678/10000
8000/8000 [==============================] - 1s 88us/step - loss: 128.4895 - val_loss: 107.1656
Epoch 2679/10000
8000/8000 [==============================] - 1s 91us/step - loss: 126.1355 - val_loss: 136.5933
Epoch 2680/10000
8000/8000 [==============================] - 1s 72us/step - loss: 137.1412 - val_loss: 114.9390
Epoch 2681/10000
8000/8000 [==============================] - 1s 66us/step - loss: 144.4085 - val_loss: 115.9378


8000/8000 [==============================] - 1s 70us/step - loss: 130.7905 - val_loss: 118.0292
Epoch 2746/10000
8000/8000 [==============================] - 1s 72us/step - loss: 126.7334 - val_loss: 119.5953
Epoch 2747/10000
8000/8000 [==============================] - 1s 67us/step - loss: 130.8221 - val_loss: 128.1364
Epoch 2748/10000
8000/8000 [==============================] - 1s 65us/step - loss: 133.0234 - val_loss: 119.7682
Epoch 2749/10000
8000/8000 [==============================] - 1s 65us/step - loss: 135.5017 - val_loss: 149.5200
Epoch 2750/10000
8000/8000 [==============================] - 1s 71us/step - loss: 145.7077 - val_loss: 128.1845
Epoch 2751/10000
8000/8000 [==============================] - 1s 70us/step - loss: 135.3547 - val_loss: 115.2390
Epoch 2752/10000
8000/8000 [==============================] - 1s 66us/step - loss: 133.4807 - val_loss: 124.9136
Epoch 2753/10000
8000/8000 [==============================] - 1s 67us/step - loss: 128.2455 - val_loss: 140.8732


8000/8000 [==============================] - 1s 69us/step - loss: 129.4186 - val_loss: 121.6082
Epoch 2818/10000
8000/8000 [==============================] - 1s 63us/step - loss: 129.4252 - val_loss: 152.4556
Epoch 2819/10000
8000/8000 [==============================] - 1s 65us/step - loss: 129.3230 - val_loss: 110.0833
Epoch 2820/10000
8000/8000 [==============================] - 1s 67us/step - loss: 132.8118 - val_loss: 109.0228
Epoch 2821/10000
8000/8000 [==============================] - 1s 68us/step - loss: 132.7737 - val_loss: 112.5044
Epoch 2822/10000
8000/8000 [==============================] - 1s 68us/step - loss: 145.6954 - val_loss: 118.9652
Epoch 2823/10000
8000/8000 [==============================] - 1s 73us/step - loss: 138.7456 - val_loss: 125.6257
Epoch 2824/10000
8000/8000 [==============================] - 1s 73us/step - loss: 135.2529 - val_loss: 112.4896
Epoch 2825/10000
8000/8000 [==============================] - 1s 72us/step - loss: 189.3118 - val_loss: 112.9522


8000/8000 [==============================] - 1s 67us/step - loss: 130.4888 - val_loss: 169.1198
Epoch 2890/10000
8000/8000 [==============================] - 1s 68us/step - loss: 128.9042 - val_loss: 125.1492
Epoch 2891/10000
8000/8000 [==============================] - 1s 66us/step - loss: 127.2655 - val_loss: 114.9866
Epoch 2892/10000
8000/8000 [==============================] - 1s 68us/step - loss: 128.6128 - val_loss: 119.2722
Epoch 2893/10000
8000/8000 [==============================] - 1s 65us/step - loss: 129.5271 - val_loss: 134.4655
Epoch 2894/10000
8000/8000 [==============================] - 1s 64us/step - loss: 127.5534 - val_loss: 125.0502
Epoch 2895/10000
8000/8000 [==============================] - 1s 66us/step - loss: 134.9353 - val_loss: 114.7426
Epoch 2896/10000
8000/8000 [==============================] - 1s 83us/step - loss: 140.0132 - val_loss: 110.0732
Epoch 2897/10000
8000/8000 [==============================] - 1s 93us/step - loss: 245.8358 - val_loss: 287.0542


8000/8000 [==============================] - 1s 66us/step - loss: 151.1921 - val_loss: 170.2529
Epoch 2962/10000
8000/8000 [==============================] - 1s 69us/step - loss: 156.1178 - val_loss: 112.5910
Epoch 2963/10000
8000/8000 [==============================] - 1s 67us/step - loss: 124.0967 - val_loss: 113.9937
Epoch 2964/10000
8000/8000 [==============================] - 1s 67us/step - loss: 140.4193 - val_loss: 112.5590
Epoch 2965/10000
8000/8000 [==============================] - 1s 66us/step - loss: 127.5001 - val_loss: 110.7075
Epoch 2966/10000
8000/8000 [==============================] - 1s 63us/step - loss: 125.6562 - val_loss: 141.5107
Epoch 2967/10000
8000/8000 [==============================] - 1s 67us/step - loss: 125.6786 - val_loss: 115.5578
Epoch 2968/10000
8000/8000 [==============================] - 1s 67us/step - loss: 150.6553 - val_loss: 116.3271
Epoch 2969/10000
8000/8000 [==============================] - 1s 69us/step - loss: 144.0355 - val_loss: 116.9473


In [4]:
# Build the oos prediction list and calculate the error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = np.sqrt(metrics.mean_squared_error(oos_pred,oos_y))
print("Final, out of sample score (RMSE): {}".format(score)) 

Final, out of sample score (RMSE): 11.61795711517334


In [ ]:

#pred = model.predict(x_test)

# Measure RMSE error.  RMSE is common for regression.
#score = np.sqrt(metrics.mean_squared_error(pred,y_test))
#print("Final score (RMSE): {}".format(score))

#chart_regression(pred.flatten(),y_test)
#chart_regression(pred.flatten(),y_test,sort=False)

In [5]:
model.load_weights(filename_checkpoint)

df_test = pd.read_csv(filename_test,na_values=['NA','?'])

extract_and_encode_features(df_test)

ids_test = df_test['id']
df_test.drop('id',1,inplace=True)

names_test = df_test['name']
df_test.drop('name',1,inplace=True)

x_submit = df_test.as_matrix().astype(np.float32)
pred_submit = model.predict(x_submit)


In [6]:
cost = [n if n > 0 else n * -1 for n in pred_submit[:,0]]
df_submit = pd.DataFrame({'id': ids_test,'cost': cost})
df_submit = df_submit[['id', 'cost']]
df_submit.to_csv(filename_submit, index=False)